In [2]:
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}
import geotrellis.raster.io.geotiff._
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{GeoTiff, SinglebandGeoTiff}
import geotrellis.spark.io.hadoop._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import geotrellis.raster.{CellType, DoubleArrayTile}
import geotrellis.raster._
import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.util.StatCounter
import scala.sys.process.Process
import sys.process._

In [3]:
val cmd = "hadoop fs -copyFromLocal " + "/data/local/home/parrot/minio2/EVI_experiment_1_SOS" + " /user/hadoop"
Process(cmd)!

cmd = hadoop fs -copyFromLocal /data/local/home/parrot/minio2/EVI_experiment_1_SOS /user/hadoop


0

In [4]:
 
val pattern: String = "tif"
var filepath: String = "hdfs:///user/hadoop/EVI_experiment_1_SOS"
val tiles_RDD = sc.hadoopGeoTiffRDD(filepath, pattern).values
var grids_RDD :RDD[Array[Double]] = sc.emptyRDD
  

Waiting for a Spark session to start...

pattern = tif
filepath = hdfs:///user/hadoop/EVI_experiment_1_SOS
tiles_RDD = MapPartitionsRDD[2] at values at <console>:63
grids_RDD = EmptyRDD[3] at emptyRDD at <console>:64


EmptyRDD[3] at emptyRDD at <console>:64

## Group the pixels from all images  on pixel positions

In [5]:
    

var grids_RDD: RDD[Array[Double]] = tiles_RDD.map(m => m.toArrayDouble())

var indexedValues: RDD[Array[(Int, Double)]] =  grids_RDD.map(m => m.zipWithIndex.map{case (e,v) => (v,e)})
indexedValues.partitions

var flattenValues : RDD[(Int, Double)] = indexedValues.flatMap(m => m).filter( m => !m._2.isNaN)

var groupedValues : RDD[(Int, Iterable[Double])] = flattenValues.groupByKey(1000)





grids_RDD = MapPartitionsRDD[4] at map at <console>:66
indexedValues = MapPartitionsRDD[5] at map at <console>:68
flattenValues = MapPartitionsRDD[7] at filter at <console>:71
groupedValues = ShuffledRDD[8] at groupByKey at <console>:73


ShuffledRDD[8] at groupByKey at <console>:73

## Calculate Average

In [6]:


val averagedValues = groupedValues.mapValues(value =>
StatCounter(value).mean).map(m => (m._1.toLong, Math.rint(m._2)))

averagedValues = MapPartitionsRDD[10] at map at <console>:71


MapPartitionsRDD[10] at map at <console>:71

## Calculate Std

In [7]:
val stdevduration = groupedValues.mapValues(value =>
StatCounter(value).stdev).map(m => (m._1.toLong, Math.rint(m._2)))

stdevduration = MapPartitionsRDD[12] at map at <console>:71


MapPartitionsRDD[12] at map at <console>:71

### Use Template

In [8]:

var num_cols_rows :(Int, Int) = (0, 0)

var tile0 = (tiles_RDD.take(1))(0)
num_cols_rows = (tile0.cols,tile0.rows)

var filepath2 = "hdfs:///user/hadoop/evi_new_template.tif"
//EVI
// var filepath = "hdfs:///user/hadoop/evi_template.tif"
//Since it is a single GeoTiff, it will be a RDD with a tile.
var geoTiff : RDD[(ProjectedExtent, Tile)] = sc.hadoopGeoTiffRDD(filepath2)
// var proExtents_RDD : RDD[ProjectedExtent] = geoTiff.keys
var tile_my : RDD[Tile]= geoTiff.values


var proExtents_RDD : RDD[ProjectedExtent] = geoTiff.keys
var extents_withIndex = proExtents_RDD.zipWithIndex().map{case (e,v) => (v,e)}
var projected_extent = (extents_withIndex.filter(m => m._1 == 0).values.cache.collect())(0)


var grids_RDD = tile_my.map(m => m.toArrayDouble())
var bands_withIndex = grids_RDD.zipWithIndex().map { case (v, i) => (i, v) }
var grid0 : RDD[(Long, Double)] = bands_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}.cache







num_cols_rows = (6500,3000)
tile0 = DoubleConstantNoDataArrayTile([D@5f35aba,6500,3000)
num_cols_rows = (6500,3000)
filepath2 = hdfs:///user/hadoop/evi_new_template.tif
geoTiff = MapPartitionsRDD[14] at mapPartitions at HadoopGeoTiffRDD.scala:109
tile_my = MapPartitionsRDD[15] at values at <console>:78
proExtents_RDD = MapPartitionsRDD[16] at keys at <console>:81
extents_withIndex = MapPartitionsRDD[18] at map at <console>:82


projected_extent: geotrellis.vector.Projecte...


MapPartitionsRDD[18] at map at <console>:82

## Produce GeoTiff

In [10]:
 var result_2: RDD[(Long, (Double, Option[Double]))] = grid0.leftOuterJoin(averagedValues) //averagedValues, stdevduration
    // result_2.take(20).foreach(println)

    //TODO: 1. If value in original .tiff  , but None in seasonality ==> set NaN for the corresponding index
var result: RDD[(Long, Double)] = result_2.map { 
    case (index, (value_org, value_pheno)) => {if (value_pheno.isDefined)  (index,value_pheno.get)
                                               else (index, Double.NaN)  }
}


var season = DoubleArrayTile(result.sortBy(_._1).map(_._2).collect(),num_cols_rows._1, num_cols_rows._2)

var geoTif = new SinglebandGeoTiff(season, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))
var output = "/data/local/home/parrot/minio2/" + "average_1" + ".tif"
GeoTiffWriter.write(geoTif, output)    


[Stage 19:==================================================> (973 + 17) / 1000]

result_2 = MapPartitionsRDD[41] at leftOuterJoin at <console>:90
result = MapPartitionsRDD[42] at map at <console>:94
season = DoubleConstantNoDataArrayTile([D@d53c0cb,6500,3000)
geoTif = SinglebandGeoTiff(DoubleConstantNoDataArrayTile([D@d53c0cb,6500,3000),Extent(-124.995535714211, 23.209821428584302, -66.95982142841754, 49.9955357142925),geotrellis.proj4.CRS$$anon$3@41d0d1b7,Tags(Map(),List()),GeoTiffOptions(geotrellis.raster.io.geotiff.Striped@21ad1100,geotrellis.raster.io.geotiff.compression.DeflateCompression$@1354f44,1,None))
output = /data/local/home/parrot/minio2/average_1.tif


/data/local/home/parrot/minio2/average_1.tif